In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from glob import glob
from matplotlib import pyplot as plt
from tensorflow import keras
from keras.preprocessing import image
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import Image, display
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input, Lambda
from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

from tensorflow.keras.applications import ResNet50

### Splitting folder into train, test, val folder

In [ ]:
# skip this if you've already run it

import splitfolders

input_folder = "pisang" #Enter Input Folder
output = "dataset2" #Enter Output Folder

splitfolders.ratio(input_folder, output=output, seed=1, ratio=(0.6,0.2,0.2))

#### Check image 

In [ ]:
listOfImageNames = ['dataset/train/pisang-kulit-luka/IMG_0003.jpg',
                   'dataset/train/pisang-kulit-tidak-luka/IMG_0022.jpg']

labels = ['Pisang kulit luka', 'Pisang kulit tidak luka']
for i,imageName in enumerate(listOfImageNames):
    print(labels[i])
    display(Image(filename=imageName))
    print("\n")

In [ ]:
imgs = cv2.imread("dataset/train/pisang-kulit-luka/IMG_0003.jpg")

In [ ]:
imgs.shape

### Data augmentation

For the documentation of data augment, please follow this link https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [ ]:
train_data_gen = ImageDataGenerator(
                    rescale = 1./255,
                    shear_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=True,
                    zoom_range=0.2,
                    rotation_range = 20,
                    width_shift_range= 0.2,
                    height_shift_range= 0.2)

validate_data_gen = ImageDataGenerator(
                    rescale = 1./255)

In [ ]:
train_data =  train_data_gen.flow_from_directory(
                        "dataset2/train",
                        target_size=(128,128),
                        batch_size=32,
                        class_mode='categorical')

validate_data = validate_data_gen.flow_from_directory(
                        "dataset2/val",
                        target_size=(128,128),
                        batch_size=32,
                        class_mode='categorical')

In [ ]:
for data_batch, labels_batch in train_data:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

## Building Model

### Follow the link below for more detailed explanation

#### Model
- CNN : https://medium.com/@draj0718/convolutional-neural-networks-cnn-architectures-explained-716fb197b243 <br>
- VGG16 : https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16 | https://keras.io/api/applications/vgg/ <br>
- Resnet50 : https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/ResNet50 | https://keras.io/api/applications/resnet/ <br>
- EarlyStopping : https://keras.io/api/callbacks/early_stopping/ <br>
- Adam optimizer : https://keras.io/api/optimizers/adam/ | https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam <br>
- Metric accuracy : https://keras.io/api/metrics/accuracy_metrics/#accuracy-class <br>
- Layers : https://keras.io/api/layers/


### CNN Model

In [ ]:
# CNN Model
modelcnn = Sequential()

# 1st layer CNN
modelcnn.add(Conv2D(filters=32, kernel_size=5, activation='relu', input_shape=[128,128,3]))
modelcnn.add(MaxPool2D(pool_size=2,padding='same'))
modelcnn.add(BatchNormalization())
# modelcnn.add(Dropout(0.2))

# 2nd layer CNN
modelcnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
modelcnn.add(MaxPool2D(pool_size=2,padding='same'))
modelcnn.add(BatchNormalization())
# modelcnn.add(Dropout(0.2))

# 3rd layer CNN
modelcnn.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
modelcnn.add(MaxPool2D(pool_size=2,padding='same'))
modelcnn.add(BatchNormalization())
# modelcnn.add(Dropout(0.2))

# 4th layer CNN
modelcnn.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
modelcnn.add(MaxPool2D(pool_size=2,padding='same'))
modelcnn.add(BatchNormalization())
modelcnn.add(Dropout(0.2))

modelcnn.add(Flatten())
modelcnn.add(Dense(515,activation='relu'))
modelcnn.add(Dense(2,activation='softmax'))

modelcnn.summary()

In [ ]:
# Get a visual view of the architecture
keras.utils.plot_model(modelcnn, to_file='arsitektur-model/modelcnn.png', show_shapes=True, show_dtype=True, 
                       show_layer_names=True, expand_nested=True,
                       dpi=75, show_layer_activations=True,
)

In [ ]:
# Create earlystopping for callback
es = EarlyStopping(monitor='val_loss', patience=3, mode='min')

In [ ]:
# Compile the model
modelcnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit the model
modelcnn.fit(train_data,
          verbose=1,
          validation_data=validate_data,
          batch_size=64,
          epochs=15)
#           callbacks=[es])

In [ ]:
# Creating dataframe of losses and accuracies of model
losses = modelcnn.history.history
df_loss = pd.DataFrame(losses)

In [ ]:
df_loss.sort_values(by='val_accuracy',ascending=False).head()

In [ ]:
# Visualizing the performance of model
pd.DataFrame(modelcnn.history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.gca().set_ylim(0, 1)
plt.title('Accuracy vs Epoch Plot of the CNN Model')
plt.show()

In [ ]:
# Evaluasi model
loss, accuracy= modelcnn.evaluate(test_data)
print(f'Accuracy: {accuracy*100}')

In [ ]:
# Save model
modelcnn.save('model/modelcnn.h5')

#### #2

In [ ]:
# CNN Model
modelcnn2 = Sequential()

# 1st layer CNN
modelcnn2.add(Conv2D(filters=32, kernel_size=5, activation='relu', input_shape=[128,128,3]))
modelcnn2.add(MaxPool2D(2,2))

# 2nd layer CNN
modelcnn2.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
modelcnn2.add(MaxPool2D(2,2))

# 3rd layer CNN
modelcnn2.add(Conv2D(filters=62, kernel_size=3, activation='relu'))
modelcnn2.add(MaxPool2D(2,2))

# 4th layer CNN
modelcnn2.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
modelcnn2.add(MaxPool2D((2,2), strides=2))
modelcnn2.add(Dropout(0.3))

modelcnn2.add(Flatten())
modelcnn2.add(Dense(512,activation='relu'))
modelcnn2.add(Dense(2,activation='softmax'))

modelcnn2.summary()

In [ ]:
modelcnn2.compile(loss = 'categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
# Fit the model
modelcnn2.fit(
      train_data,
      steps_per_epoch=len(train_data),
      epochs=15,
      validation_data=validate_data,
      validation_steps=len(validate_data))

In [ ]:
# Visualizing the performance of model
pd.DataFrame(modelcnn2.history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.gca().set_ylim(0, 1)
plt.title('Accuracy vs Epoch Plot of the CNN Model')
plt.show()

In [ ]:
modelcnn2.evaluate(test_data)

### VGG16 Model

In [ ]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=(128, 128, 3), weights='imagenet', include_top=False)
vgg.trainable = False

In [ ]:
# our layers - you can add more if you want
inputs = keras.Input(shape=(128, 128, 3))
X = vgg(inputs,training=False)
X = MaxPool2D()(X)
X = Flatten()(X)
X = Dense(128, activation='relu')(X)
X = Dense(2, activation='softmax')(X)

# create a model object
modelvgg16 = Model(inputs=inputs, outputs=X)

In [ ]:
# view the structure of the model
modelvgg16.summary()

In [ ]:
# Get a visual view of the architecture
keras.utils.plot_model(modelvgg16, to_file='arsitektur-model/modelvgg16.png', show_shapes=True, show_dtype=True, 
                       show_layer_names=True, expand_nested=True, 
                       dpi=75, show_layer_activations=True,
)

In [ ]:
# compile the model
modelvgg16.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [ ]:
# fit the model
modelvgg16.fit(train_data,
             validation_data=validate_data,
             epochs=15,
             steps_per_epoch=len(train_data),
             validation_steps=len(test_data),
#              callbacks=[es]
            )

In [ ]:
# Creating dataframe of losses and accuracies of model
losses = modelvgg16.history.history
df_loss = pd.DataFrame(losses)

In [ ]:
df_loss.sort_values(by='val_accuracy',ascending=False).head()

In [ ]:
# Visualizing the performance of model
pd.DataFrame(modelvgg16.history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.gca().set_ylim(0, 1)
plt.title('Accuracy vs Epoch Plot of the VGG16 Model')
plt.show()

In [ ]:
# Evaluasi model
loss, accuracy= modelvgg16.evaluate(test_data)
print(f'Accuracy: {accuracy*100}')

In [ ]:
# Save model
modelvgg.save('model/modelvgg16.h5')

### ResNet50 Model

In [ ]:
# add preprocessing layer to the front of VGG
rsnet = ResNet50(input_shape=(150, 150, 3), weights='imagenet', include_top=False)
# don't train existing weights
for layer in rsnet.layers:
  layer.trainable = False

# our layers - you can add more if you want
X = MaxPool2D()(rsnet.output)
X = Flatten()(X)
X = Dense(512, activation='relu')(X)
X = Dense(2, activation='softmax')(X)

# create a model object
modelresnet50 = Model(inputs=rsnet.input, outputs=X)

In [ ]:
# view the structure of the model
modelresnet50.summary()

In [ ]:
# Get a visual view of the architecture
keras.utils.plot_model(modelresnet50, to_file='arsitektur-model/modelresnet50.png', show_shapes=True, show_dtype=True, 
                       show_layer_names=True, expand_nested=True,
                       dpi=75, show_layer_activations=True,
)

In [ ]:
# tell the model what cost and optimization method to use
modelresnet50.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
# fit the model
modelresnet50.fit(train_data,
              validation_data=validate_data,
              epochs=15,
              steps_per_epoch=len(train_data),
              validation_steps=len(validate_data),
#               callbacks=[es]
                 )

In [ ]:
# Creating dataframe of losses and accuracies of model
losses = modelresnet50.history.history
df_loss = pd.DataFrame(losses)

In [ ]:
df_loss.sort_values(by='val_accuracy',ascending=False).head()

In [ ]:
# Visualizing the performance of model
pd.DataFrame(modelresnet50.history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.gca().set_ylim(0, 1)
plt.title('Accuracy vs Epoch Plot of the VGG16 Model')
plt.show()

In [ ]:
# Evaluasi model
loss, accuracy= modelresnet50.evaluate(test_data)
print(f'Accuracy: {accuracy*100}')

In [ ]:
modelresnet50.save('model/modelresnet50.h5')

### Predict

In [ ]:
class_names =  ["Pisang Kulit Luka", "Pisang Kulit Tidak Luka"]

image_path = "dataset/test/pisang-kulit-tidak-luka/IMG_0034.jpg"
new_img = image.load_img(image_path, target_size=(150, 150))
img = image.img_to_array(new_img)/255
img = np.expand_dims(img, axis=0)

images = np.vstack([img])

prediction = model.predict(images, batch_size=10)
predictions = np.argmax(prediction,axis=1)

print('prediksi: {}'.format(class_names[predictions[0]]))
print('persentase prediksi: {:.2f} %'.format(np.max(prediction)*100))
print('\n')
for i in range(len(class_names)):
    print('prediksi:\t{}'. format(class_names[i]))
    print("persentase:\t{:.2f} %".format(prediction[0][i]*100))

plt.imshow(new_img)

### Load Model

In [ ]:
# For load the model you can use the code
from keras.models import load_model 
model = load_model('model/modelcnn.h5')

### Confusion Matrix 

In [ ]:
# We need to recreate our test generator with shuffle = false
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        "dataset2/test",
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

class_labels = test_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

#Confution Matrix and Classification Report
Y_pred = modelcnn2.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
# import libraries
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# create confusion matrix
cm = confusion_matrix(test_generator.classes, y_pred)

print('Classification Report\n')
target_names = list(class_labels.values())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
import seaborn as sns

# Menampilkan confusion matrix menggunakan heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Prediksi")
plt.ylabel("Label")
plt.show()